In [1]:
import sys
sys.path.append("../")
sys.path.append("../../")

In [2]:
import numpy as np
from Onsager_calc_db import *
import onsager.crystal as crystal
from states import *
from stars import *
from vector_stars import *
from representations import *
import time
import pickle
from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
from matplotlib import pyplot as plt
import scipy.stats as spstats

/usr/local/lib/python3.6/dist-packages/onsager-1.3.3-py3.6.egg/onsager/cluster.py:835: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.


In [ ]:
# make a BCC lattice
latt = np.array([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]) * 0.2831
Fe = crystal.Crystal(latt, [[np.array([0., 0., 0.]), np.array([0.5, 0.5, 0.5])]], ["Fe"])
# Now give it the orientations - for BCC it's [110]
o = np.array([1.,1.,0.])/np.linalg.norm(np.array([1.,1.,0.]))*0.126*2
famp0 = [o.copy()]
family = [famp0]
pdbcontainer_fe = dbStates(Fe, 0, family)
mdbcontainer_fe = mStates(Fe, 0, family)
jset0, jset2 = pdbcontainer_fe.jumpnetwork(0.26, 0.01, 0.01), mdbcontainer_fe.jumpnetwork(0.26, 0.01, 0.01)

In [4]:
# Modify jnet0
jnet0 = jset0[0]
jnet0_indexed = jset0[1]
# Let's try to sort the jumps according to smallest net atomic displacement
def sortkey(entry):
    jmp = jnet0[entry][0]
    or1 = pdbcontainer_fe.iorlist[jmp.state1.iorind][1]
    or2 = pdbcontainer_fe.iorlist[jmp.state2.iorind][1]
    dx = disp(pdbcontainer_fe, jmp.state1, jmp.state2)
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices = []
for jt, jlist in enumerate(jnet0):
#     if np.allclose(jnet0_indexed[jt][0][1], z):
#         continue
    indices.append(jt)

ind_sort = sorted(indices, key=sortkey)[:3]
len(ind_sort)

3

In [5]:
# we'll consider all the pure dumbbell jumps mentioned in the database,
# They are: 60-degree roto-translation, rigid translation and 60-degree on-site rotation
for ind in ind_sort:
    jmp = jnet0[ind][0]
    print(jmp)
    print(np.dot(Fe.lattice, jmp.state2.R))
    print()

Jump object:
Initial state:
	dumbbell : (i, or) index = 0, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 5, lattice vector = [0 1 0]
Jumping from c1 = 1 to c2 = 1

[-0.14155  0.14155 -0.14155]

Jump object:
Initial state:
	dumbbell : (i, or) index = 0, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 5, lattice vector = [0 0 0]
Jumping from c1 = 1 to c2 = -1

[0. 0. 0.]

Jump object:
Initial state:
	dumbbell : (i, or) index = 0, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 0, lattice vector = [0 1 0]
Jumping from c1 = 1 to c2 = -1

[-0.14155  0.14155 -0.14155]



In [6]:
pdbcontainer_fe.iorlist

[(0, array([-0.17819091,  0.17819091,  0.        ])),
 (0, array([ 1.78190909e-01,  1.01947883e-17, -1.78190909e-01])),
 (0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17])),
 (0, array([ 1.01947883e-17, -1.78190909e-01, -1.78190909e-01])),
 (0, array([ 0.        , -0.17819091,  0.17819091])),
 (0, array([0.17819091, 0.        , 0.17819091]))]

In [7]:
# take the jumps we want.
jset0new = ([jnet0[i] for i in ind_sort], [jnet0_indexed[i] for i in ind_sort])

In [8]:
# Now, we modify the mixed dumbbell jumpnetwork.
# We'll include all the jumps there are in the database of Messina et. al.
# They are: 60 deg rot-trans, rigid translation.
jnet2 = jset2[0]
jnet2_indexed = jset2[1]
# Let's try to sort the jumps according to closest distance
# we don't want only the rotational jumps as listed.

def sortkey2(entry):
    jmp = jnet2[entry][0]
    or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = disp(mdbcontainer_fe, jmp.state1, jmp.state2)
    # c1 and c2 are always +1 for mixed dumbbell jumps.
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices2 = []
indices_rot = []
for jt, jlist in enumerate(jnet2):
    if np.allclose(jnet2_indexed[jt][0][1], z):
        jmp = jlist[0]

        or1 = mdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
        or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
        
        if np.allclose(np.dot(or1,or2)/(np.linalg.norm(or1)*np.linalg.norm(or2)), np.cos(np.pi/3.)):
#             print("got 60 deg jump at {}".format(jt))
            indices_rot.append(jt)
            continue
        elif np.allclose(np.dot(or1,or2), 0.):
#             print("got 90 deg jump at {}".format(jt))
            indices_rot.append(jt)
            continue
        else:
            continue
    indices2.append(jt)
ind_sort2 = sorted(indices2, key=sortkey2)[:2]
indices2all = ind_sort2[:2]
# indices2all.append(indices_rot[0])
print(indices2all, indices_rot)

[7, 11] [18, 19]


In [10]:
# check if we have the correct type of jumps
for ind in indices2all:
    jmp = jnet2[ind][0]
    print(jmp)
#     print(np.dot(Fe.lattice, jmp.state2.R))
    print()

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 1, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [-1 -1 -1]
	dumbbell : (i, or) index = 11, lattice vector = [-1 -1 -1]
Jumping from c1 = 1 to c2 = 1

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 1, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [-1 -1 -1]
	dumbbell : (i, or) index = 10, lattice vector = [-1 -1 -1]
Jumping from c1 = 1 to c2 = 1



In [16]:
print(mdbcontainer_fe.iorlist[11])
print(mdbcontainer_fe.iorlist[1])
print(mdbcontainer_fe.iorlist[10])

(0, array([-1.01947883e-17,  1.78190909e-01,  1.78190909e-01]))
(0, array([ 1.78190909e-01,  1.01947883e-17, -1.78190909e-01]))
(0, array([-1.78190909e-01, -1.01947883e-17,  1.78190909e-01]))


In [13]:
# take only the lowest displacement jump
jset2new = ([jnet2[i] for i in indices2all], [jnet2_indexed[i] for i in indices2all])
len(jset0new[0]), len(jset2new[0])

(3, 2)

In [15]:
# re-initialize the calculator.
# Now, we construct the Onsager calculator with these non-local jump sets.
start = time.time()
onsagercalculator = dumbbellMediated(pdbcontainer_fe, mdbcontainer_fe, jset0new, jset2new, 0.26,
                                     0.01, 0.01, 0.01, NGFmax=4, Nthermo=3)
print("onsager calculator initiation time = {}".format(time.time() - start))

initializing thermo
initializing kin
initializing NN
built shell 1: time - 0.058695077896118164
built shell 2: time - 1.4776544570922852
grouped states by symmetry: 1.056215763092041
built mixed dumbbell stars: 0.00060272216796875
built jtags2: 0.0006031990051269531
built mixed indexed star: 0.00560760498046875
building star2symlist : 0.00010228157043457031
building bare, mixed index dicts : 0.0002155303955078125
2NN Shell initialization time: 3.5749449729919434

generating thermodynamic shell
built shell 1: time - 0.04499411582946777
built shell 2: time - 1.6029636859893799
built shell 3: time - 7.245916843414307
grouped states by symmetry: 5.797067642211914
built mixed dumbbell stars: 0.0004105567932128906
built jtags2: 0.0005810260772705078
built mixed indexed star: 0.0055921077728271484
building star2symlist : 0.00012230873107910156
building bare, mixed index dicts : 0.00019240379333496094
thermodynamic shell generated: 20.18230104446411
Total number of states in Thermodynamic Shel

### Next, we must extract the necessary omega43 data

In [17]:
jnet43 = onsagercalculator.jnet43
jnet43_indexed = onsagercalculator.jnet43_indexed
# Let's try to sort the jumps according to closest distance
# We assume that association and dissociation occurs via Johnson mechanism.

def sortkey3(entry):
    jmp = jnet43[entry][0] # This is an omega4 jump
    if not jmp.c2 == -1:
        print(c2)
    or1 = pdbcontainer_fe.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer_fe.iorlist[jmp.state2.db.iorind][1]
    dx = disp4(pdbcontainer_fe, mdbcontainer_fe, jmp.state1, jmp.state2)
    # remember that c2 is -1 for an omega4 jump
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx - or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(jmp.c2*or2/2.)
    return dx1+dx2+dx3

z = np.zeros(3)
indices43 = []
for jt, jlist in enumerate(jnet43):
    if np.allclose(jnet43_indexed[jt][0][1], z):
        continue
    indices43.append(jt)    
ind_sort43 = sorted(indices43, key=sortkey3)[:1]
print(ind_sort43)

[2]


In [19]:
print(jnet43[ind_sort43[0]][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 2, lattice vector = [ 0  0 -1]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 10, lattice vector = [0 0 0]
Jumping from c1 = -1 to c2 = -1


In [20]:
pdbcontainer_fe.iorlist

[(0, array([-0.17819091,  0.17819091,  0.        ])),
 (0, array([ 1.78190909e-01,  1.01947883e-17, -1.78190909e-01])),
 (0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17])),
 (0, array([ 1.01947883e-17, -1.78190909e-01, -1.78190909e-01])),
 (0, array([ 0.        , -0.17819091,  0.17819091])),
 (0, array([0.17819091, 0.        , 0.17819091]))]

In [21]:
mdbcontainer_fe.iorlist

[(0, array([-0.17819091,  0.17819091,  0.        ])),
 (0, array([ 1.78190909e-01,  1.01947883e-17, -1.78190909e-01])),
 (0, array([ 1.78190909e-01,  1.78190909e-01, -1.01947883e-17])),
 (0, array([ 1.01947883e-17, -1.78190909e-01, -1.78190909e-01])),
 (0, array([ 0.17819091, -0.17819091,  0.        ])),
 (0, array([ 0.        , -0.17819091,  0.17819091])),
 (0, array([-1.78190909e-01, -1.78190909e-01,  1.01947883e-17])),
 (0, array([0.17819091, 0.        , 0.17819091])),
 (0, array([ 0.        ,  0.17819091, -0.17819091])),
 (0, array([-0.17819091,  0.        , -0.17819091])),
 (0, array([-1.78190909e-01, -1.01947883e-17,  1.78190909e-01])),
 (0, array([-1.01947883e-17,  1.78190909e-01,  1.78190909e-01]))]

In [24]:
print(onsagercalculator.crys)

#Lattice:
  a1 = [0.14155 0.14155 0.14155]
  a2 = [-0.14155  0.14155 -0.14155]
  a3 = [-0.14155 -0.14155  0.14155]
#Basis:
  (Fe) 0.0 = [0. 0. 0.]


In [25]:
onsagercalculator.regenerate43(ind_sort43)

In [26]:
with open('Fe_Cu_Ni_Si.pkl','wb') as fl:
    pickle.dump(onsagercalculator,fl)